In [31]:
import joblib
model = joblib.load('norm_model.sav')

In [32]:
model.feature_names_in_

array(['Emotion_Joy', 'Emotion_Sadness', 'Emotion_Anger',
       'Emotion_Disgust', 'Emotion_Fear', 'Emotion_Surprise',
       'Emotion_Contempt', 'Emotion_Neutral', 'Gender_enc'], dtype=object)

In [33]:
# Import pandas library 
import pandas as pd 
# Load test set 
test = pd.read_csv('Norm-PHO-Binary-Test.csv') 

In [35]:
print("\nNorm-PHO-Binary.csv Column Names:")
print(list(test.columns))


Norm-PHO-Binary.csv Column Names:
['Gender', 'Emotion_Joy', 'Emotion_Sadness', 'Emotion_Anger', 'Emotion_Disgust', 'Emotion_Fear', 'Emotion_Surprise', 'Emotion_Contempt', 'Emotion_Neutral', 'Depression']


In [40]:
print("\nNorm-PHO-Binary.csv Column Names:")
print(list(test.columns))


Norm-PHO-Binary.csv Column Names:
['Gender', 'Emotion_Joy', 'Emotion_Sadness', 'Emotion_Anger', 'Emotion_Disgust', 'Emotion_Fear', 'Emotion_Surprise', 'Emotion_Contempt', 'Emotion_Neutral', 'Depression', 'Gender_enc']


In [41]:
# Import LabelEncoder 
from sklearn import preprocessing 
# Create LabelEncoder 
le = preprocessing.LabelEncoder() # Convert string categories into numbers 
test['Gender'] = le.fit_transform(test['Gender']) 
test['Depression'] = le.fit_transform(test['Depression']) 

In [42]:
# Indicate the target column 
y_test = test['Depression'] 
# Indicate the columns that will serve as features 
x_test = test.drop('Depression', axis = 1) 

In [43]:
# Import pickle 
import pickle 
# Specify the file name of your saved model 
# filename='freq_model.sav' for Freq-PHO-Binary 
# filename='norm_model.sav' for Norm-PHO-Binary 
filename='norm_model.sav' 
# Load the model 
best_model = pickle.load(open(filename, 'rb')) 

In [48]:
import pandas as pd
from sklearn import preprocessing
import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings

# Suppress warnings related to scikit-learn version mismatch when loading the model
warnings.filterwarnings('ignore', category=UserWarning, message='Trying to unpickle estimator SVC from version')

# --- 1. Load Test Set ---
test_norm = pd.read_csv('Norm-PHO-Binary-Test.csv')

# --- 2. Prepare Test Data (Label Encoding) ---

# Create LabelEncoder to convert string categories to numeric values
le = preprocessing.LabelEncoder()

# Apply LabelEncoder to 'Gender' and 'Depression'
test_norm['Gender'] = le.fit_transform(test_norm['Gender'])
test_norm['Depression'] = le.fit_transform(test_norm['Depression'])

# --- 3. Separate target and features ---
y_test = test_norm['Depression'] # Target
x_test = test_norm.drop('Depression', axis=1) # Features

# --- 4. FIX: Rename feature column to match the trained model's expectation ---
if 'Gender' in x_test.columns:
    # Model expects 'Gender_enc', so rename the Label Encoded 'Gender' column
    x_test = x_test.rename(columns={'Gender': 'Gender_enc'})

# Ensure feature order matches the model's training features for consistency
expected_features = ['Emotion_Joy', 'Emotion_Sadness', 'Emotion_Anger', 'Emotion_Disgust',
                     'Emotion_Fear', 'Emotion_Surprise', 'Emotion_Contempt',
                     'Emotion_Neutral', 'Gender_enc']
x_test = x_test[expected_features]

# --- 5. Load the Best Model ---
filename_norm = 'norm_model.sav'
with open(filename_norm, 'rb') as file:
    best_model = pickle.load(file)

# --- 6. Predict the target for the test dataset ---
test_predict = best_model.predict(x_test)

# --- 7. Compute and Print Metrics ---
print("--- Results for Best Norm Model (SVC) on Test Set ---")
print("Accuracy (Test): ", accuracy_score(y_test, test_predict))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, test_predict))

print("\nClassification Report (Precision, Recall, F1-Score):")
print(classification_report(y_test, test_predict))

--- Results for Best Norm Model (SVC) on Test Set ---
Accuracy (Test):  0.654320987654321

Confusion Matrix:
[[33 14]
 [14 20]]

Classification Report (Precision, Recall, F1-Score):
              precision    recall  f1-score   support

           0       0.70      0.70      0.70        47
           1       0.59      0.59      0.59        34

    accuracy                           0.65        81
   macro avg       0.65      0.65      0.65        81
weighted avg       0.65      0.65      0.65        81

